In [1]:
import networkx as nx
import numpy as np
import json
import pandas as pd
import pickle

title_embeddings = pd.read_csv("/Users/gabesmithline/Desktop/gnn_project/data/title_embeddings.csv", index_col=0)
abstract_embeddings = pd.read_csv("/Users/gabesmithline/Desktop/gnn_project/data/abstract_embeddings.csv", index_col=0)
title_embeddings = title_embeddings.transpose()
abstract_embeddings = abstract_embeddings.transpose()

with open("/Users/gabesmithline/Desktop/gnn_project/data/paper_source_trace_valid_wo_ans.json", "r") as f:
    paper_source = json.load(f)
with open("/Users/gabesmithline/Desktop/gnn_project/data/paper_source_trace_train_ans.json", "r") as f:
    paper_source_train = json.load(f)
paper_source = paper_source + paper_source_train

G = nx.Graph()

for paper in paper_source:
    paper_id = paper["_id"]
    if paper_id in title_embeddings.index and paper_id in abstract_embeddings.index:
        G.add_node(paper_id, 
                   title=paper["title"], 
                   authors=paper["authors"],
                   year=paper["year"] if "year" in paper else [],
                   venue=paper["venue"] if "venue" in paper else [],
                   title_embeddings=title_embeddings.loc[paper_id].tolist(),
                   abstract_embeddings=abstract_embeddings.loc[paper_id].tolist()
                   )  
    else:
        G.add_node(paper_id, 
                   title=paper["title"],
                   authors=paper["authors"], 
                   year=paper["year"] if "year" in paper else [],
                   venue=paper["venue"] if "venue" in paper else [],
                   title_embeddings=[],
                   abstract_embeddings=[]
                   )

for paper in paper_source:
    for source in paper["references"]:
        if source in G: # Only add edge if both nodes exist in graph 
            G.add_edge(paper["_id"], source)


with open('/Users/gabesmithline/Desktop/gnn_project/data/adj_matrix.pkl', 'wb') as f:
    pickle.dump(G, f)

